[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](<https://colab.research.google.com/github/MauPos04/AI_NOTEBOOKS/blob/main/Web%20Scrap%20and%20Translate%20Using%20Transformers.ipynb>)

# Web Scraping y Aplicación de Transformers

## WebScraping

In [38]:
%pip install requests
%pip install beautifulsoup4

In [39]:
# importacion de librerias
from bs4 import BeautifulSoup
import requests

In [40]:
# URL de la pagina a scrapear
URL = "https://www.diariobitcoin.com/ia/alibaba-redefine-ia-nuevo-modelo-qwq-32b-optimiza-rendimiento-en-ia-con-menos-recursos/"
page = requests.get(URL)

In [41]:
print(page.text)

        <!DOCTYPE html><html lang="es">
        	<head>

		<meta name='robots' content='index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1' />
	<style>img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }</style>
	<script id="cookieyes" type="text/javascript" src="https://cdn-cookieyes.com/client_data/b55f6124d8f412f4e556506c/script.js"></script>
	<!-- This site is optimized with the Yoast SEO plugin v24.6 - https://yoast.com/wordpress/plugins/seo/ -->


	<title>Alibaba redefine la IA: Nuevo modelo Qwq-32b optimiza rendimiento con menos recursos - DiarioBitcoin</title>
	<link rel="canonical" href="https://www.diariobitcoin.com/ia/alibaba-redefine-ia-nuevo-modelo-qwq-32b-optimiza-rendimiento-en-ia-con-menos-recursos/" />
	<meta property="og:locale" content="es_ES" />
	<meta property="og:type" content="article" />
	<meta property="og:title" content="Alibaba redefine la IA: Nuevo modelo Qwq-32b optimiza rendimiento con menos recur

In [42]:
# Creacion de un objeto BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

In [43]:
# Extraccion de los elementos de la pagina
results = soup.find_all("div", class_="post-content")

In [44]:
# iteración para seleccionar los resultados de la búsqueda (título, empresa y ubicación)
for new_element in results:
  news_element = new_element.find("div", class_="entry-content")

In [45]:
# extracción del texto de la noticia a partir de la etiqueta <hr/>
news_element_str = str(news_element)

soup = BeautifulSoup(news_element_str, 'html.parser')
hr_tag = soup.find('hr')
content_after_hr = hr_tag.find_all_next(string=True)
extracted_text = '\n'.join(content_after_hr)

In [46]:
# limpieza del texto extraído de la noticia y eliminación de la marca de agua
final_text = 'Imagen original de DiarioBitcoin'
Text = extracted_text.strip().replace("\n", "").split(final_text)[0]
print(Text)

El equipo de Qwen de Alibaba ha lanzado QwQ-32B, un modelo de 32 mil millones de parámetros que promete redefinir la eficiencia de rendimiento en tareas complejas. Diseñado para mejorar el razonamiento a través del aprendizaje por refuerzo (RL), QwQ-32B ya está disponible en varias plataformas abiertas.Este modelo emerge en un panorama en constante evolución, donde los modelos tradicionales han revelado limitaciones significativas en escalabilidad y eficiencia. Las leyes de escalado han provocado un interés renovado en los modelos de razonamiento grande, capaces de aplicar inferencia y autorreflexión para tareas desafiantes.Alcance y aplicaciones empresariales de QwQ-32BDirigido a líderes empresariales y profesionales de la tecnología, QwQ-32B representa un cambio potencial en cómo las organizaciones pueden aprovechar la IA para la toma de decisiones y la innovación técnica. Su capacidad para proporcionar conocimientos más estructurados y detallados lo hace esencial para el análisis de

In [53]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

## Summarize

In [51]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'csebuetnlp/mT5_multilingual_XLSum'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt).to(device)

def generate_summary_csebuetnlp(text):
    inputs = tokenizer(
        [text],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [52]:
generate_summary_csebuetnlp(Text)

'El equipo de Qwen de Alibaba ha lanzado Quuq-32B, un modelo de 32 mil millones de parámetros que promete redefinir la eficiencia de rendimiento en tareas complejas.'

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt).to(device)

def generate_summary_bart_large(text):
    inputs = tokenizer(
        [text],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [50]:
generate_summary_bart_large(Text)

'QwQ-32B promete redefinir la eficiencia de rendimiento en tareas complejas. Diseñado para mejorar el razonamiento a través del aprendizaje por refuerzo (RL) Modelo ya está disponible en varias plataformas abiertas.'

## Translation

In [60]:
model_id = "ModelSpace/GemmaX2-28-2B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

text = f"Translate this from Spanish to English:\nSpanish: {Text}\nEnglish:"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Translate this from Spanish to English:
Spanish: El equipo de Qwen de Alibaba ha lanzado QwQ-32B, un modelo de 32 mil millones de parámetros que promete redefinir la eficiencia de rendimiento en tareas complejas. Diseñado para mejorar el razonamiento a través del aprendizaje por refuerzo (RL), QwQ-32B ya está disponible en varias plataformas abiertas.Este modelo emerge en un panorama en constante evolución, donde los modelos tradicionales han revelado limitaciones significativas en escalabilidad y eficiencia. Las leyes de escalado han provocado un interés renovado en los modelos de razonamiento grande, capaces de aplicar inferencia y autorreflexión para tareas desafiantes.Alcance y aplicaciones empresariales de QwQ-32BDirigido a líderes empresariales y profesionales de la tecnología, QwQ-32B representa un cambio potencial en cómo las organizaciones pueden aprovechar la IA para la toma de decisiones y la innovación técnica. Su capacidad para proporcionar conocimientos más estructurados 